<h1><center>ChBE 4746/6746 Spring 2022 Skill Checks</center></h1>
<h3><center>Instructions: Skill Checks will be included in every homework to help you practice your Python and Pyomo skills. These will include short coding exercises that will teach you skills that you will need to solve the problems of the homework sets. </center></h3>


## SC1

There are multiple ways you can create an optimization problem in Pyomo, some more efficient than others (especially when a problem is large). In this exercise, you will have to create three versions of the same simple optimization problem, using non-indexed variables vs indexed variables and pyomo vs python expressions. The problem you will be formulating and solving is following: 

$$ \max x_\mathrm{1} + x_\mathrm{2}  $$

<center>s.t.</center> $$ 2x_\mathrm{1} - x_\mathrm{2} \le 10 $$

$$ -20 \le x_\mathrm{1},x_\mathrm{2} \le 20 $$



### SC1.a. Unindexed variables, explicit Pyomo expressions
The most intuitive way, if you're not used to using indices is to name each variable with a different name and explicitly write out all of the equations. If you have 1-2 variables, this could be ok. Complete the Pyomo formulation below:

In [1]:
from pyomo.environ import *
import numpy as np

m = ConcreteModel()

# declare variables
m.x1 = Var(within=Reals, bounds = (-20, 20))
m.x2 = Var(within=Reals, bounds = (-20, 20))

# add constraints
m.con1 = Constraint(expr= 2*m.x1 - m.x2  <= 10)

# set objective function
m.obj = Objective(expr= m.x1 + m.x2, sense = maximize)

solver = SolverFactory('glpk')
solver.solve(m)
m.pprint()
print("Optimal x: {:.2f} {:.2f} ".format(value(m.x1), value(m.x2)))
print("Optimal objective: {:.2f}".format(value(m.obj)))

2 Var Declarations
    x1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :   -20 :  15.0 :    20 : False : False :  Reals
    x2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :   -20 :  20.0 :    20 : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : x1 + x2

1 Constraint Declarations
    con1 : Size=1, Index=None, Active=True
        Key  : Lower : Body      : Upper : Active
        None :  -Inf : 2*x1 - x2 :  10.0 :   True

4 Declarations: x1 x2 con1 obj
Optimal x: 15.00 20.00 
Optimal objective: 35.00


In [2]:
assert np.isclose(value(m.x1), 15.00), "Not optimal solution"
assert np.isclose(value(m.x2), 20.00), "Not optimal solution"
assert np.isclose(value(m.obj), 35.00), "Not optimal objective value"

### SC1.b. Indexed Variables, explicit Pyomo Expressions
What would happen to above approach if you had 10's or 100's or 1000's variables? The better approach is to use sets and indices, so let's get familiar with this in the following skill check. Complete the Pyomo formulation and ensure you get the same answer as above.

In [3]:
from pyomo.environ import *
m = ConcreteModel()

# set: python primitive list
I = [1, 2]

# declare variables
m.x = Var(I, within=Reals, bounds = (-20,20))

# constraints: python sum()
m.con1 = Constraint(expr= 2*m.x[1] - m.x[2] <= 10)


# objective: python sum()
m.obj = Objective(expr= m.x[1] + m.x[2], sense = maximize)

solver = SolverFactory('glpk')
solver.solve(m)
print("Optimal x: ", end="")
for i in I:
    print("{:.2f}".format(value(m.x[i])), end=" ")
print("\nOptimal objective: {:.2f}".format(value(m.obj)))

Optimal x: 15.00 20.00 
Optimal objective: 35.00


In [4]:
assert type(m.x) == pyomo.core.base.var.IndexedVar, "Not an indexed variable"
assert np.isclose(value(m.x[1]), 15.00), "Not optimal solution"
assert np.isclose(value(m.x[2]), 20.00), "Not optimal solution"
assert np.isclose(value(m.obj), 35.00), "Not optimal objective value"

### SC1c: Indexed Variables, parameters, equations, Pyomo expressions
In SC1b, we took advantage of a set to declare one variable instead of two, but the expressions in the constraint and objective were still explicit. We can take a step further and take advantage of indexed sets within equations. Complete the Pyomo formulation below and make sure you get the same answer as in a and b.

In [5]:
from pyomo.environ import *
m = ConcreteModel()

# set: python primitive list
I = [1, 2]

# define parameters:
A = {1:2, 2:-1}
B = 10

# declare variables
m.x = Var(I, within=Reals, bounds = (-20,20))

# constraints: python sum()
m.con1 = Constraint(expr= summation(A, m.x) <= B)

# objective: python sum()
m.obj = Objective(expr= summation(m.x), sense = maximize)

solver = SolverFactory('glpk')
solver.solve(m)
print("Optimal x: ", end="")
for i in I:
    print("{:.2f}".format(value(m.x[i])), end=" ")
print("\nOptimal objective: {:.2f}".format(value(m.obj)))


Optimal x: 15.00 20.00 
Optimal objective: 35.00


In [6]:
assert np.isclose(value(m.x[1])**m.con1() - value(m.x[2])**m.con1(), -9663349609375.0), "Not optimal solution"

### SC1d: Indexed Variables, parameters, equations, Python expressions
The only difference in the following exercise is that we will not use the "Summation" expression that is a Pyomo expression, but we will use Python's "sum" to express the constraints and objective. The purpose of this exercise is to show you that both ways are understood by Pyomo, and it is up to you to pick what works best for you. Complete following example and make sure you get the same optimum as before:

In [7]:
from pyomo.environ import *
m = ConcreteModel()

# set: python primitive list
I = [1, 2]

# define parameters:
A = {1:2, 2:-1}
B = 10

# declare variables
m.x = Var(I, within=Reals, bounds = (-20,20))

# constraints: python sum()
m.con1 = Constraint(expr= sum(A[i] * m.x[i] for i in I) <= B)

# objective: python sum()
m.obj = Objective(expr= sum(m.x[i] for i in I), sense = maximize)

solver = SolverFactory('glpk')
solver.solve(m)
print("Optimal x: ", end="")
for i in I:
    print("{:.2f}".format(value(m.x[i])), end=" ")
print("\nOptimal objective: {:.2f}".format(value(m.obj)))


Optimal x: 15.00 20.00 
Optimal objective: 35.00


In [8]:
assert np.isclose(value(m.x[1])**m.con1() - value(m.x[2])**m.con1(), -9663349609375.0), "Not optimal solution"

### SC1.e Indexed Variables/Parameters/Equations, Use of `rules` for constraints and objective
Sometimes you will have a general type of equation (or else "rule") that could be used to represent multiple constraints, each time with different parameter values. Other times, a constraint may be a bit more complex than just a single equation, and it would be convenient to be able to use Python definitions (def) to define this (or you may already have the def written in some other Python file you already have, and want to use it as is). This is shown in the following example. Complete Pyomo formulation and verify that you get the same result as above.

In [1]:
from pyomo.environ import *
import numpy as np
m = ConcreteModel()

# set: python primitive list
I = [1, 2]

# define parameters:
A = {1:2, 2:-1}
B = 10

# declare variables
m.x = Var(I, within=Reals, bounds = (-20,20))

# add constraints with python functions and 'rule'
def con1(pA,pB):
    # within the python function, the model name should correspond to the argument
    return pA[1]*m.x[1] + pA[2]*m.x[2] <=pB
    #return sum(pA[i] * model.x[i] for i in I) <= pB

m.con1 = Constraint(rule=con1(A,B))

# objective: 
m.obj = Objective(expr= sum(m.x[i] for i in I), sense = maximize)

solver = SolverFactory('glpk')
solver.solve(m)
print("Optimal x: ", end="")
for i in I:
    print("{:.2f}".format(value(m.x[i])), end=" ")
print("\nOptimal objective: {:.2f}".format(value(m.obj)))

Optimal x: 15.00 20.00 
Optimal objective: 35.00


In [10]:
assert np.isclose(B*m.con1()/(value(m.obj)**(value(m.x[1])/value(m.x[2]))), 6.949426511707964), "Not optimal solution"

### SC1.f: Indexed Variables/Parameters/Equations, Pyomo expressions, Pyomo indices and Parameter declarations
In the following example, we will practice yet again another variation of the formulation. Here, we define the index as a Pyomo set and the parameters as Pyomo parameters. This formulation has advantages when one wants to do advanced operations with sets and change values of parameters, and we will see more examples of this as the course progresses. For now, it just suffices for you to know that this is also a viable formulation. Complete the formulation and make sure you get the same result as before.

In [11]:
from pyomo.environ import *
m = ConcreteModel()

# Pyomo Set
m.i = Set(initialize=[1,2])

# create Pyomo Parameters
m.A = Param(m.i, initialize={1: 2, 2: -1})
m.B = Param(initialize = 10)

# declare variables
m.x = Var(m.i, within=Reals, bounds = (-20,20))

# constraints: python sum()
m.con1 = Constraint(expr= summation(m.A, m.x) <= m.B)

# objective: python sum()
m.obj = Objective(expr= summation(m.x), sense = maximize)

solver = SolverFactory('glpk')
solver.solve(m)
print("Optimal x: ", end="")
for i in m.i:
    print("{:.2f}".format(value(m.x[i])), end=" ")
print("\nOptimal objective: {:.2f}".format(value(m.obj)))

Optimal x: 15.00 20.00 
Optimal objective: 35.00


In [12]:
assert np.isclose(value(m.B)*value(m.obj)/(m.con1()**(value(m.x[1])/value(m.x[2]))), 62.239779351362294), "Not optimal solution"